In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from PIL import Image

# Step 1: Load the Dataset
def load_data(images_folder, masks_folder):
    image_files = os.listdir(images_folder)
    mask_files = os.listdir(masks_folder)
    images = []
    masks = []
    for img_file in image_files:
        img_path = os.path.join(images_folder, img_file)
        img = np.array(Image.open(img_path).convert('RGB'))
        images.append(img)

    for mask_file in mask_files:
        mask_path = os.path.join(masks_folder, mask_file)
        mask = np.array(Image.open(mask_path).convert('L'))
        masks.append(mask)

        # mask_file = img_file.split('.')[0] + '_mask.png'  # Assuming mask files are named similarly
        # mask_path = os.path.join(masks_folder, mask_file)
        # mask = np.array(Image.open(mask_path).convert('L'))  # Convert to grayscale
        # masks.append(mask)

    return np.array(images), np.array(masks)

# Step 2: Preprocess the Data
def preprocess_data(images, masks):
    # Resize images and masks, normalize pixel values, and prepare the data for training
    images = images.astype('float32') / 255.0
    masks = masks.astype('float32') / 255.0  # Assuming masks are in [0, 255] range
    return images, masks

# Step 3: Define the U-Net Model
def unet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

    # Define the U-Net architecture
    # Contracting Path
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Expansive Path
    up6 = layers.Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(conv5)
    conv4_resized = tf.image.resize(conv4, tf.shape(up6)[1:3], method=tf.image.ResizeMethod.BILINEAR)
    up6 = layers.concatenate([up6, conv4_resized], axis=3)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = layers.Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    conv3_resized = tf.image.resize(conv3, tf.shape(up7)[1:3], method=tf.image.ResizeMethod.BILINEAR)
    up7 = layers.concatenate([up7, conv3_resized], axis=3)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
    conv2_resized = tf.image.resize(conv2, tf.shape(up8)[1:3], method=tf.image.ResizeMethod.BILINEAR)
    up8 = layers.concatenate([up8, conv2_resized], axis=3)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv8)
    conv1_resized = tf.image.resize(conv1, tf.shape(up9)[1:3], method=tf.image.ResizeMethod.BILINEAR)
    up9 = layers.concatenate([up9, conv1_resized], axis=3)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = layers.Conv2D(3, 1, activation='softmax')(conv9)  # Assuming 3 output classes
    print("final layer", outputs.shape)

    original_shape_outputs = tf.image.resize(outputs, input_shape[:2])
    print("final layer modified layer", original_shape_outputs.shape)

    # Create the model
    model = models.Model(inputs=inputs, outputs=original_shape_outputs)
    return model

# Step 4: Compile the Model
def compile_model(model):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 5: Train the Model
def train_model(model, images, masks, epochs=10, batch_size=32, validation_split=0.2):
    model.fit(images, masks, epochs=epochs, batch_size=batch_size, validation_split=validation_split)

# Main function
if __name__ == "__main__":
    # Set the paths to your dataset folders
    images_folder = r'C:\Users\Admin\Documents\Naufal\Program\U-Net\train\images'
    masks_folder = r'C:\Users\Admin\Documents\Naufal\Program\U-Net\train\masks'

    # Load dataset
    images, masks = load_data(images_folder, masks_folder)
    print("shape images", images.shape)
    print("shape masks", masks.shape)

    # Preprocess the data
    images, masks = preprocess_data(images, masks)
    print("~shape images", images.shape)
    print("~shape masks", masks.shape)

    # Define input shape
    input_shape = images[0].shape
    print("shape masks", images[0].shape)

    # Initialize the model
    model = unet(input_shape)

    # Compile the model
    compile_model(model)

    # Train the model
    train_model(model, images, masks)


shape images (59, 200, 101, 3)
shape masks (59, 200, 101)
~shape images (59, 200, 101, 3)
~shape masks (59, 200, 101)
shape masks (200, 101, 3)
final layer (None, 192, 96, 3)
final layer modified layer (None, 200, 101, 3)
Epoch 1/10
2/2 [==============================] - 2s 658ms/step - loss: nan - accuracy: 0.2622 - val_loss: nan - val_accuracy: 0.8163
Epoch 2/10
2/2 [==============================] - 1s 328ms/step - loss: nan - accuracy: 0.8000 - val_loss: nan - val_accuracy: 0.8163
Epoch 3/10
2/2 [==============================] - 1s 327ms/step - loss: nan - accuracy: 0.8000 - val_loss: nan - val_accuracy: 0.8163
Epoch 4/10
2/2 [==============================] - 1s 323ms/step - loss: nan - accuracy: 0.8000 - val_loss: nan - val_accuracy: 0.8163
Epoch 5/10
2/2 [==============================] - 1s 319ms/step - loss: nan - accuracy: 0.8000 - val_loss: nan - val_accuracy: 0.8163
Epoch 6/10
2/2 [==============================] - 1s 318ms/step - loss: nan - accuracy: 0.8000 - val_loss: n

In [13]:
import cv2
import numpy as np

# Read the image file using OpenCV
image = cv2.imread(r'C:\Users\Admin\Documents\Naufal\Program\U-Net\train\masks\3_img_png.rf.86c077a0fb6d191e5dc359eeaff17904.jpg')

# Convert the image to a NumPy array
image_array = np.array(image)

# Now you can work with the image data as a NumPy array


In [14]:
import numpy as np
unique_values, counts = np.unique(image_array, return_counts=True)

# Print the counts for each unique value
for value, count in zip(unique_values, counts):
    print(f"Value {value}: Count {count}")

Value 0: Count 46416
Value 1: Count 627
Value 2: Count 375
Value 3: Count 210
Value 4: Count 54
Value 5: Count 33
Value 6: Count 6
Value 7: Count 3
Value 8: Count 3
Value 10: Count 3
Value 11: Count 9
Value 12: Count 3
Value 13: Count 3
Value 14: Count 3
Value 15: Count 3
Value 20: Count 6
Value 21: Count 3
Value 26: Count 3
Value 30: Count 9
Value 32: Count 6
Value 35: Count 3
Value 36: Count 3
Value 37: Count 12
Value 39: Count 15
Value 43: Count 3
Value 45: Count 6
Value 48: Count 3
Value 51: Count 3
Value 54: Count 3
Value 55: Count 3
Value 59: Count 3
Value 63: Count 6
Value 64: Count 3
Value 65: Count 3
Value 68: Count 3
Value 69: Count 3
Value 70: Count 3
Value 74: Count 3
Value 76: Count 3
Value 89: Count 3
Value 90: Count 3
Value 91: Count 6
Value 92: Count 6
Value 93: Count 6
Value 94: Count 6
Value 95: Count 12
Value 96: Count 3
Value 97: Count 6
Value 101: Count 6
Value 105: Count 3
Value 106: Count 3
Value 108: Count 3
Value 109: Count 6
Value 122: Count 9
Value 123: Count